To use this notebook interactively, install the `bash_kernel` Python package with either:
* `pip install bash_kernel`
* `conda install -c conda-forge bash_kernel`

# OSS Module 01 - Git Basics

# Contents
- [1: What is version control?](#1:-What-is-version-control?)
- [2: Git and GitHub Setup](#2:-Git-and-GitHub-setup)
- [3: A Git project directory](#3:-A-Git-project-directory)
- [4: A Git workflow from start to finish](#4:-A-Git-workflow-from-start-to-finish)
  * [4.1 : Creating a repo and committing files](#4.1:-Creating-a-repo-and-committing-files)

# 1: What is version control?

# Section 2: Git and GitHub setup

# 3: A Git project directory

## Parts of a Git Project Directory

The contents of a Git repo are stored on your computer as normal files in a normal directory.  In Git terminology, these files are the **working tree**.  You can edit these files freely with your usual text editors and IDEs.  

On your computer, changes to the files are stored in the **local repository** (or local **database**).  Soon, we will learn how to store (or **commit**) new versions of the files; and how to retrieve (or **checkout**) previous versions of the files.  

A small project directory is shown below. 
* The database and metadata are stored in the hidden `.git/` directory.  You can treat this as a black box, and your only interaction with the database and metadata should be via `git` commands.
* `my_program.py` and `README.md` are files you are actively working on.  You definitely want to record these in the local repo.
* `__pycache__` contains intermediate bytecode from running your Python program.  You probably do *not* want to record these in the local repo...

``` Console
~/my_git_project$ ls -la
total 95
drwxr-xr-x  4 gburdell  staff   6 Jan 25 12:33 .
drwx--x--x 43 gburdell  staff  97 Jan 25 12:33 ..
drwxr-xr-x  7 gburdell  staff  10 Jan 25 12:33 .git
-rw-r--r--  1 gburdell  staff  69 Jan 25 12:32 my_program.py
drwxr-xr-x  2 gburdell  staff   3 Jan 25 12:33 __pycache__
-rw-r--r--  1 gburdell  staff  51 Jan 25 12:33 README.md
```

## Tracked vs. Untracked Files

Git does not automatically add files from the working tree to the repo.  Instead, the user must explicitly specify which files are stored in the repo (**tracked**) and which are not (**untracked**).  It's important to consider which files to *not* track.  

In a Python project, you should not track byte code (`*.pyc` and `*.pyo` in `__pycache__` directories) since they will be automatically regenerated whenever someone else runs it with a different operating system or Python version.  Tracking them will just take up unnecessary space in the database. 

For C/C++ and other compiled languages, you should not track build artifacts (`*.o` files, libraries, executables) for a simlar reason.  These artifacts will usually not work on a different operating system.

# 4: Working with a Git repo

In this section, we will create a local repo, add files, and work remote repo on GitHub

## 4.1: Creating a repo and committing files

### `git init`: Creating a local repo

The `git init` command creates a local repo in the current directory.  Let's create a new directory, `~/git-workflow` and initialize a repo inside it.  

(You can use a directory with a different name and location, if you like.)

In [3]:
mkdir ~/git-workflow
cd ~/git-workflow
git init

Initialized empty Git repository in /Users/gburdell/git-workflow/.git/


You can see the `.git/` directory was created for the database and metadata.  

In [4]:
ls -lha

total 0
drwxr-xr-x    3 gburdell  staff    96B May  7 20:36 .
drwxr-xr-x+ 191 gburdell  staff   6.0K May  7 20:36 ..
drwxr-xr-x    9 gburdell  staff   288B May  7 20:36 .git


### `git status`: Show the state of the working tree

`git status` shows the state of tracked files, untracked files, and the database. Right now, the working tree is empty, and the database is empty, so there's not much to report.  

We already have a **branch** called `main`, which was created when the repo was initialized.  We'll cover branches in detail soon.  For now, you can think of them like branches of a family tree that describe your project's history.  

In [5]:
git status

On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)


### Creating new files

To create and edit files in your working tree, you can use any text editor.  In `~/git_workflow`, let's create two files.

* First, create a file named `README.md` file with these contents.  You might notice a mistake, but don't correct it right now! :)

``` text
# Git Workflow

This is an example repo for the GT OSPO VSIP Spring 2024 Program.  
```

* Then, let's create a Python source file named `my_abs.py` with these contents. We will be making changes to `my_abs.py` throughout this demo.  

``` python
def my_abs(x):
    if x < 0:
        return -x
    else:
        return x
```

### Untracked files

`git status` reports that the new files are **untracked**.  Recall that untracked files are not recorded in repo's database, and that Git does not automatically track new files.  Git helpfully suggests that we should track them with the `git add` command, and we'll do that soon.  But first let's discuss the possible **states** for files in your working tree.  Knowing these states can help you troubleshoot many issues when you're working with your repo. 

In [6]:
git status

On branch main

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md
	my_abs.py

nothing added to commit but untracked files present (use "git add" to track)


### The state of files in a working tree

Files in the working tree are either tracked or untracked:

* **Untracked**: No versions of the file are stored in the repo's database
* **Tracked**:  One or more versions of the file are in the database

Additionally, changes in tracked files are always in one of three states:

* **Unmodified**:  The file in the working is tree are up-to-date with the database.  Git sometimes refers to these changes as "up-to-date".  
* **Modfied**:  The file has changes that have not yet been stored in the database.  Furthermore, the user hasn't specified that these changes will be stored in the next database update.  
* **Staged**: The file's changes that will be stored in the next database update.  Git sometimes refers to these as "to be committed".  


<img src="img/lifecycle.png" width="800" />


(Image credit:  Scott Chacon and Ben Straub.  Pro Git, [Section 2.2](https://git-scm.com/book/en/v2/Git-Basics-Recording-Changes-to-the-Repository))

### `git add`: Stage new changes



In our working copy of `git_workflow`, we have two untracked files: `README.md` and `my_abs.py`.  To record their changes, we need two steps (as shown in the [diagram above](#The-state-of-files-in-a-working-tree)):

1. **Add the files:**  This changes the files' states from "Untracked" to "Staged".  At this point, the changes are *ready* to be stored in the database but *are not yet stored*.  
2. **Commit**:  This changes their state from "Staged" to "Unmodified".  At this point, the changes have actually been stored in the database.

To accomplish Step 1 (Untracked ➞ Staged), we use `git add`.  Its usage is:

```
git add <file1> [<file2> ...]
```

In [7]:
git add README.md my_abs.py

Now `git status` shows the files have changes that are "to be committed".  This means the changes are staged for the next database update.  But remember that the database has **not** actually been updated.

In [8]:
git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   README.md
	new file:   my_abs.py



### Modifying a staged file

What happens when you've staged a file, but before you commit it, you realize you need to fix something?  For example, in `README.md`, we made a typo.  We wrote "Spring 2024" instead of "Summer 2024".  

Let's correct `README.md` with our text editor, and then look at the `git status`:

In [9]:
git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   README.md
	new file:   my_abs.py

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   README.md



Git tells us:
* `README.md` and `my_abs.py` have changes that are staged ("to be committed")
* `README.md` also has changes that are not staged

How can one file have both staged and unstaged changes?  The reason is that `git add` stages the state of the file **at the exact moment** you run `git add`.  So if you run `git add`, and make additional changes afterwards (like changing "Spring" to "Summer"), then those additional changes are not automatically staged.  

### Staging new content (again)

To fix this, we'll run `git add` again to stage the new changes ("Spring" to "Summer").  You can see that Git suggests this, too, when it says: 'use "git add <file>..." to update what will be committed'

In [10]:
git add README.md

Now we see that all the changes in `README.md` have been staged, since there are no "Changes not staged for commit" anymore.  

In [11]:
git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   README.md
	new file:   my_abs.py



### `git commit`: Updating the database

Finally, we'll use `git commit` to record the previously-staged changes to the database.  

You can use the `-m` option to specify a **commit message** on the command line.  This is a short message that lets other humans know what changes you've made.  Many projects have conventions about what info should go into a commit message.  Ask your project manager for details.  

(If you do not use `-m`, a text editor will pop up and prompt you to enter a message.)

In [12]:
git commit -m "First commit of my_abs (no try/except yet)"

[main (root-commit) 02b19f4] First commit of my_abs (no try/except yet)
 2 files changed, 9 insertions(+)
 create mode 100644 README.md
 create mode 100644 my_abs.py


### `git log`:  Showing the repo's history

Now that we actually have information in our repo's database, we can use `git log` to show the history.  `git log` has many options to show more or less information about the history.  You can run `git help log` to see all the available options.

In [13]:
git log

commit 02b19f4848824bbacc3bce833cb33ad29a2e784d (HEAD -> main)
Author: George Burdell <gburdell@gatech.edu>
Date:   Tue May 7 20:38:23 2024 -0500

    First commit of my_abs (no try/except yet)


## 4.2: Working with Remotes

Up to this point, we've only made changes to the **local** repo on our computer.  We haven't touched GitHub at all.  Now, we are going to create a **remote** repo on GitHub and upload our local repo to it.

### Creating a remote repo

From the front page of [GitHub](https://github.com), you can create a new repo by clicking on the the "+" button on the top menu bar.  You can also go directly to https://github.com/new

<img src="img/working-with-remote-01.png" width="500" />

You should now see the "Create a new repository" page.  Since we are uploading an existing repository (instead of creating a new one), we'll only need a few options.

* **Repository template**:  GitHub has starter templates (with directories and some boilerplate code) for common types of projects.  We'll select "No template".
* **Owner**:  The repo's owner can be an individual account or an organization (a group of accounts).  A GitHub organization is a powerful tool for collaborating and managing permissions, and more likely than not, you'll be working in an organization for your VSIP project.  For this tutorial, let's use your individual GitHub account.
* **Repository name**: This will be the last part of the repo's URL.  When someone clones your repo, it will also be the default name for the project directory.  Let's give it the same name as your existing project directory.
* **Visibility**: You can make your repo either Public or Private, whichever you prefer
* **Initialize this repository** and below:  Since we are uploading an existing repo, we should not create a `README`, `.gitignore`, or license.

When you're finished, click the "Create repository" button

<img src="img/working-with-remote-02.png" width="500" />

You will now be taken to your repo's webpage, and you should see the following info.  

<img src="img/working-with-remote-03.png" width="600" />

In [20]:
git remote add origin git@github.com:GeorgeBurdell/git-workflow.git

In [21]:
git remote -v

origin	git@github.com:GeorgeBurdell/git-workflow.git (fetch)
origin	git@github.com:GeorgeBurdell/git-workflow.git (push)


In [22]:
git push -u origin main

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 12 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 408 bytes | 408.00 KiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
To github.com:GeorgeBurdell/git-workflow.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


## Section 4.3: Downloading a repo

In [23]:
mkdir ~/somewhere-else
cd ~/somewhere-else

In [24]:
git clone git@github.com:GeorgeBurdell/git-workflow.git

Cloning into 'git-workflow'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), done.


In [26]:
cd ~/somewhere-else/git-workflow

In [30]:
ls -la

total 16
drwxr-xr-x   5 gburdell  staff  160 May  7 20:43 .
drwxr-xr-x   3 gburdell  staff   96 May  7 20:43 ..
drwxr-xr-x  12 gburdell  staff  384 May  7 20:43 .git
-rw-r--r--   1 gburdell  staff   84 May  7 20:43 README.md
-rw-r--r--   1 gburdell  staff   75 May  7 20:43 my_abs.py


In [31]:
git log

commit 02b19f4848824bbacc3bce833cb33ad29a2e784d (HEAD -> main, origin/main, origin/HEAD)
Author: George Burdell <gburdell@gatech.edu>
Date:   Tue May 7 20:38:23 2024 -0500

    First commit of my_abs (no try/except yet)


# Section 4.4: Adding more changes

In [32]:
cd ~/git-workflow

## Modifying an unmodified file

Copy/paste this into my_abs.py


``` python
import math

def my_abs(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except TypeError:
        return math.nan
```

In [33]:
git diff

diff --git a/my_abs.py b/my_abs.py
index fc89810..64431e6 100644
--- a/my_abs.py
+++ b/my_abs.py
@@ -1,6 +1,11 @@
+import math
+
 def my_abs(x):
-    if x < 0:
-        return -x
-    else:
-        return x
+    try:
+        if x < 0:
+            return -x
+        else:
+            return x
+    except TypeError:
+        return math.nan
 


In [34]:
git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   my_abs.py

no changes added to commit (use "git add" and/or "git commit -a")


In [35]:
git commit -am "Added try/except"

[main c3c4498] Added try/except
 1 file changed, 9 insertions(+), 4 deletions(-)


In [36]:
git log --oneline --graph --branches --remotes 

* c3c4498 (HEAD -> main) Added try/except
* 02b19f4 (origin/main) First commit of my_abs (no try/except yet)


In [37]:
git push

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 12 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 376 bytes | 376.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To github.com:GeorgeBurdell/git-workflow.git
   02b19f4..c3c4498  main -> main


# Section 6: Branches

## Our branches

<img src="img/branches-01.png" width="400" />

## Checking out a commit

In [46]:
git checkout 02b19f4

Note: switching to '02b19f4'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 02b19f4 First commit of my_abs (no try/except yet)


In [47]:
git log --oneline --graph --branches --remotes 

* c3c4498 (origin/main, main) Added try/except
* 02b19f4 (HEAD) First commit of my_abs (no try/except yet)


<img src="img/branches-02.png" width="400" />

In [48]:
cat my_abs.py

def my_abs(x):
    if x < 0:
        return -x
    else:
        return x



## Checking out a branch

In [49]:
git checkout main

Previous HEAD position was 02b19f4 First commit of my_abs (no try/except yet)
Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [50]:
git log --oneline --graph --branches --remotes 

* c3c4498 (HEAD -> main, origin/main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-03.png" width="400" />

In [51]:
cat my_abs.py

import math

def my_abs(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except TypeError:
        return math.nan



## Creating a topic branch

In [52]:
git checkout -b types

Switched to a new branch 'types'


<img src="img/branches-04.png" width="400" />

In [53]:
git log --oneline --graph --branches --remotes 

* c3c4498 (HEAD -> types, origin/main, main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


Copy this to `my_abs.py`

``` python
import math
import numbers

def my_abs(x):
    if isinstance(x, numbers.Real):
        if x < 0:
            return -x
        else:
            return x
    elif isinstance(x, numbers.Complex):
        return math.sqrt(
            x.real ** 2 + x.imag ** 2)
    else:
        return math.nan
```

In [54]:
git diff

diff --git a/my_abs.py b/my_abs.py
index 64431e6..bf90a6f 100644
--- a/my_abs.py
+++ b/my_abs.py
@@ -1,11 +1,15 @@
 import math
+import numbers
 
 def my_abs(x):
-    try:
+    if isinstance(x, numbers.Real):
         if x < 0:
             return -x
         else:
             return x
-    except TypeError:
+    elif isinstance(x, numbers.Complex):
+        return math.sqrt(
+            x.real ** 2 + x.imag ** 2)
+    else:
         return math.nan
 


In [55]:
git commit -am "Using type checks"

[types 864205a] Using type checks
 1 file changed, 6 insertions(+), 2 deletions(-)


In [56]:
git log --oneline --graph --branches --remotes 

* 864205a (HEAD -> types) Using type checks
* c3c4498 (origin/main, main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-05.png" width="500" />

## Making another topic branch

In [57]:
git checkout main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [58]:
git checkout -b almost-eq

Switched to a new branch 'almost-eq'


<img src="img/branches-06.png" width="500" />

Copy this to `my_abs.py`

``` python
import math

def my_abs(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except TypeError:
        return math.nan

def my_almost_eq(x, y):
    return my_abs(x - y) < 1e-16
```

In [59]:
git diff

diff --git a/my_abs.py b/my_abs.py
index 64431e6..efa815d 100644
--- a/my_abs.py
+++ b/my_abs.py
@@ -9,3 +9,6 @@ def my_abs(x):
     except TypeError:
         return math.nan
 
+def my_almost_eq(x, y):
+    return my_abs(x - y) < 1e-16
+


In [60]:
git commit -am "Added my_almost_eq"

[almost-eq 557e440] Added my_almost_eq
 1 file changed, 3 insertions(+)


In [61]:
git log --oneline --graph --branches --remotes 

* 557e440 (HEAD -> almost-eq) Added my_almost_eq
| * 864205a (types) Using type checks
|/  
* c3c4498 (origin/main, main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-07.png" width="500" />

## `git merge`: A fast-forward merge

In [62]:
git checkout main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [63]:
git log --oneline --graph --branches --remotes 

* 557e440 (almost-eq) Added my_almost_eq
| * 864205a (types) Using type checks
|/  
* c3c4498 (HEAD -> main, origin/main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-08.png" width="500" />

In [64]:
git merge types

Updating c3c4498..864205a
Fast-forward
 my_abs.py | 8 ++++++--
 1 file changed, 6 insertions(+), 2 deletions(-)


In [65]:
git log --oneline --graph --branches --remotes 

* 557e440 (almost-eq) Added my_almost_eq
| * 864205a (HEAD -> main, types) Using type checks
|/  
* c3c4498 (origin/main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-09.png" width="500" />

## `git merge`:  A merge commit

In [66]:
git merge --no-edit almost-eq

Auto-merging my_abs.py
Merge made by the 'recursive' strategy.
 my_abs.py | 3 +++
 1 file changed, 3 insertions(+)


In [67]:
git log --oneline --graph --branches --remotes 

*   0dfda78 (HEAD -> main) Merge branch 'almost-eq'
|\  
| * 557e440 (almost-eq) Added my_almost_eq
* | 864205a (types) Using type checks
|/  
* c3c4498 (origin/main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-10.png" width="500" />

In [68]:
cat my_abs.py

import math
import numbers

def my_abs(x):
    if isinstance(x, numbers.Real):
        if x < 0:
            return -x
        else:
            return x
    elif isinstance(x, numbers.Complex):
        return math.sqrt(
            x.real ** 2 + x.imag ** 2)
    else:
        return math.nan

def my_almost_eq(x, y):
    return my_abs(x - y) < 1e-16



# Branches and remotes

In [69]:
git log --oneline --graph --branches --remotes 

*   0dfda78 (HEAD -> main) Merge branch 'almost-eq'
|\  
| * 557e440 (almost-eq) Added my_almost_eq
* | 864205a (types) Using type checks
|/  
* c3c4498 (origin/main) Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


In [70]:
git push

Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 12 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 1018 bytes | 509.00 KiB/s, done.
Total 9 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 1 local object.
To github.com:GeorgeBurdell/git-workflow.git
   c3c4498..0dfda78  main -> main


In [71]:
git log --oneline --graph --branches --remotes 

*   0dfda78 (HEAD -> main, origin/main) Merge branch 'almost-eq'
|\  
| * 557e440 (almost-eq) Added my_almost_eq
* | 864205a (types) Using type checks
|/  
* c3c4498 Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-11.png" width="500" />

## Back to somewhere-else

In [72]:
cd ~/somewhere-else/git-workflow

In [73]:
git log --oneline --graph --branches --remotes 

* 02b19f4 (HEAD -> main, origin/main, origin/HEAD) First commit of my_abs (no try/except yet)


<img src="img/branches-12.png" width="400" />

In [74]:
cat my_abs.py

def my_abs(x):
    if x < 0:
        return -x
    else:
        return x



In [75]:
git fetch

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 3), reused 12 (delta 3), pack-reused 0
Unpacking objects: 100% (12/12), 1.29 KiB | 55.00 KiB/s, done.
From github.com:GeorgeBurdell/git-workflow
   02b19f4..0dfda78  main       -> origin/main


In [76]:
git log --oneline --graph --branches --remotes 

*   0dfda78 (origin/main, origin/HEAD) Merge branch 'almost-eq'
|\  
| * 557e440 Added my_almost_eq
* | 864205a Using type checks
|/  
* c3c4498 Added try/except
* 02b19f4 (HEAD -> main) First commit of my_abs (no try/except yet)


<img src="img/branches-13.png" width="500" />

In [77]:
git merge origin/main

Updating 02b19f4..0dfda78
Fast-forward
 my_abs.py | 18 +++++++++++++++---
 1 file changed, 15 insertions(+), 3 deletions(-)


In [78]:
git log --oneline --graph --branches --remotes 

*   0dfda78 (HEAD -> main, origin/main, origin/HEAD) Merge branch 'almost-eq'
|\  
| * 557e440 Added my_almost_eq
* | 864205a Using type checks
|/  
* c3c4498 Added try/except
* 02b19f4 First commit of my_abs (no try/except yet)


<img src="img/branches-14.png" width="500" />

In [79]:
cat my_abs.py

import math
import numbers

def my_abs(x):
    if isinstance(x, numbers.Real):
        if x < 0:
            return -x
        else:
            return x
    elif isinstance(x, numbers.Complex):
        return math.sqrt(
            x.real ** 2 + x.imag ** 2)
    else:
        return math.nan

def my_almost_eq(x, y):
    return my_abs(x - y) < 1e-16

